In [40]:
import yt_dlp

def download_mp4_from_youtube(url):
    # Set the options for the download
    filename = 'lecuninterview.mp4'
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]',
        'outtmpl': filename,
        'quiet': True,
    }

    # Download the video file
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(url, download=True)

url = "https://www.youtube.com/watch?v=mBjPyte2ZZo"
download_mp4_from_youtube(url)

In [41]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("lecuninterview.mp4")
print(result['text'])

C:\Users\akash\anaconda3\envs\Project_1\Lib\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 

In [ ]:
with open ('text.txt', 'w') as file:  
    file.write(result['text'])

In [42]:
with open('text.txt') as f:
    text = f.read()

In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)


In [44]:
from langchain.docstore.document import Document
texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts[:4]]

In [45]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DeepLake
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
from dotenv import load_dotenv
import os

load_dotenv("keys.env") 
ACTIVELOOP_TOKEN= os.getenv("ACTIVELOOP_TOKEN")
# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "akashghanathey"
my_activeloop_dataset_name = "langchain_course_youtube_summarizer"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://akashghanathey/langchain_course_youtube_summarizer already exists, loading from the storage


In [46]:
db.add_documents(docs)


Creating 4 embeddings in 1 batches of size 4:: 100%|█████████████████████████████████████| 1/1 [00:05<00:00,  5.48s/it]

Dataset(path='hub://akashghanathey/langchain_course_youtube_summarizer', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype     shape     dtype  compression
  -------    -------   -------   -------  ------- 
 embedding  embedding  (8, 384)  float32   None   
    id        text      (8, 1)     str     None   
 metadata     json      (8, 1)     str     None   
   text       text      (8, 1)     str     None   


['358000ca-e029-11ef-b3fe-683e26c5319e',
 '358000cb-e029-11ef-9527-683e26c5319e',
 '358000cc-e029-11ef-a137-683e26c5319e',
 '358000cd-e029-11ef-8558-683e26c5319e']

In [47]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['k'] = 4

In [48]:
from langchain.chains.summarize import load_summarize_chain
import textwrap
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langchain.chains import RetrievalQA

llm_engine=ChatOllama(
    model="deepseek-r1:14b",
    temperature=0.6
)

prompt_template ="""Use the following pieces of transcripts from a video to answer the question in bullet points and summarized. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Summarized answer in bullter points:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm_engine,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 chain_type_kwargs=chain_type_kwargs)


print( qa.invoke("Summarize the mentions of google according to their AI program") )

{'query': 'Summarize the mentions of google according to their AI program', 'result': "<think>\nAlright, let me try to figure out how to approach this query. The user provided a transcript and asked for bullet points summarizing Google's mentions in their AI program. I need to parse through the transcript carefully.\n\nFirst, I'll read through the transcript again to spot any references to Google. Scanning through, I notice two parts where Google is mentioned. In both cases, it's part of a list that includes Facebook, YouTube, and others. The context talks about how these companies use self-supervised learning techniques for content moderation systems.\n\nSo, from this, I can gather that Google uses self-supervised learning in their AI applications, specifically for tasks like content moderation. There's no mention of specific projects or the name of their AI program beyond the general technique they employ.\n\nI should structure the bullet points to reflect these findings clearly. Eac